In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

import cv2
import os

import numpy as np
import pandas as pd
import math
import random

In [ ]:
path = '../input/petfinder-pawpularity-score/'
scores = pd.read_csv(path + 'train.csv')
img_size=300
current_batch = 0
last_batch = 0
batchrates = []

In [ ]:
def encodescore(X):
    out = [0, 0, 0]
    if (X < 25):
        return out
    elif (X < 50):
        out = [0, 0, 1]
        return out
    elif (X < 75):
        out = [0, 1, 1]
        return out
    else:
        out = [1, 1, 1]
        return out

def preprocessdata(X):
    X_p = keras.applications.resnet50.preprocess_input(X)
    return X_p
        

def assemble_batch(batch_num, batch_length):
    start_index = batch_num * batch_length
    total_length = scores.shape[0]
    data = []
    if (batch_num == batches - 1):
        if (total_length % batches > 0):
            batch_length = batch_length + (total_length % batches)
    end_index = start_index + batch_length
    for x in range(start_index, end_index):
        entry = scores.iloc[x]
        try:
            img_arr = cv2.imread(os.path.join(path + 'train', entry.Id + '.jpg'))[...,::-1] 
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
            data.append([resized_arr, entry])
        except Exception as e:
            print(e)
    print(f'Batch {batch_num+1} assembled. Length: {batch_length}')
    return np.array(data, dtype=object)
    
def train_batch(z, model, batch_length, max_epochs, lastLoss=999, rMetric="loss", f=1, q=1.0, d=0.0, phase=1, ls=4): 
    global batchrates
    global current_batch
    global val_x
    global val_y
    train = assemble_batch(z, batch_length)
    current_batch = z
    x_train = []
    y_train = []
    
    if (phase == 1):     
        for feature, labels in train:
            x_train.append(feature)
            y_train.append(labels[f:])

        del train    
        
        for x in range(len(y_train)):
            y_train[x] = y_train[x].to_list()
            for y in range(len(y_train[x])):
                if (y == len(y_train[x]) - 1):
                    encoded = encodescore(y_train[x][y])
                    y_train[x][y] = encoded[0]
                    y_train[x].append(encoded[1])
                    y_train[x].append(encoded[2])
                y_train[x][y] = float(y_train[x][y])
            y_train[x] = np.array(y_train[x])
        
        x_train = np.array(x_train) / 255
        x_train = preprocessdata(x_train)
    
        y_train = np.asarray(y_train).astype('float32')
        
    elif (phase == 2):
        for feature, labels in train:
            x_train.append(feature)
            y_train.append(labels[-1:])
            
        del train 
            
        for x in range(len(y_train)):
            y_train[x] = y_train[x].to_list()
            for y in range(len(y_train[x])):
                y_train[x][y] = float(y_train[x][y])
            y_train[x] = np.array(y_train[x])
            
        x_train = np.array(x_train) / 255
        x_train = preprocessdata(x_train)
        y_train = np.asarray(y_train).astype('float32')
    
    history = model.fit(x_train,y_train,epochs = max_epochs,batch_size=ls,shuffle=True,validation_data=(val_x, val_y))   
    del x_train
    del y_train
    del history
    return model, lastLoss

  
class SoftBinaryCap(Layer):
    def __init__(self):
        super(SoftBinaryCap, self).__init__()

    def build(self, input_shape):
        
        self.w = tf.Variable(initial_value=tf.convert_to_tensor([[1.],[0.]]),
            trainable=False
        )

    def call(self, inputs):
        x = tf.matmul(inputs, self.w)
        return x
    
class ScoreRenderer(Layer):
    def __init__(self):
        super(ScoreRenderer, self).__init__()

    def build(self, input_shape):
        self.a = self.add_weight(shape=(3,),
                                 constraint=tf.keras.constraints.NonNeg(),
                               trainable=True)
        
        self.b = self.add_weight(shape=(6,),
                                 constraint=tf.keras.constraints.NonNeg(),
                               trainable=True)
        
        self.c = self.add_weight(shape=(6,),
                                 constraint=tf.keras.constraints.NonNeg(),
                               trainable=True)
        
        

    def call(self, inputs): 
        l = tf.shape(inputs)[0]  
        pos = keras.backend.map_fn(lambda i: i * self.b, inputs[:,:6])
        neg = keras.backend.map_fn(lambda i: i * self.c, inputs[:,6:12])
        x = keras.backend.map_fn(lambda i:  i * self.a, inputs[:,12:])
        x = tf.reduce_sum(x, axis=1)
        x = keras.backend.reshape(x, (l, 1))
        pos = tf.reduce_sum(pos, axis=1)
        neg = tf.reduce_sum(neg, axis=1)
        pos = keras.backend.reshape(pos, (l, 1))
        neg = keras.backend.reshape(neg, (l, 1))
        x = tf.concat([x, (pos - neg)], 1)
        return x

In [ ]:
batches = 21
batch_length = math.floor(scores.shape[0] / batches)

print(f'Total Length: {scores.shape[0]} | Batches: {batches} | Batch Length: {batch_length}')

In [ ]:
val_x = []
val_y = []
val_data = assemble_batch(batches-1, batch_length)

for feature, labels in val_data:
    val_x.append(feature)
    val_y.append(labels[1:])

del val_data    
        
for x in range(len(val_y)):
    val_y[x] = val_y[x].to_list()
    for y in range(len(val_y[x])):
        if (y == len(val_y[x]) - 1):
            encoded = encodescore(val_y[x][y])
            val_y[x][y] = encoded[0]
            val_y[x].append(encoded[1])
            val_y[x].append(encoded[2])
        val_y[x][y] = float(val_y[x][y])
    val_y[x] = np.array(val_y[x])
        
val_x = np.array(val_x) / 255
val_x = preprocessdata(val_x)
    
val_y = np.asarray(val_y).astype('float32')

In [ ]:
res_model = keras.applications.ResNet50(include_top=False, weights="../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5", input_tensor=keras.Input(shape=(img_size, img_size, 3)))

In [ ]:
for layer in res_model.layers[:143]:
    layer.trainable = False

In [ ]:
class binaryFeatureDetector(keras.Model):

  def __init__(self):
    super(binaryFeatureDetector, self).__init__()
    self.dense1 = layers.Dense(128, use_bias=False, kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.dense2 = layers.Dense(64, use_bias=False, kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.dense3 = layers.Dense(32, use_bias=False, kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.dense4 = layers.Dense(16, use_bias=False, kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.dense5 = layers.Dense(8, use_bias=False, kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.softmax = layers.Dense(2, use_bias=False, activation='softmax', kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
    self.cap = SoftBinaryCap()

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.dense5(x)
    x = self.softmax(x)
    return self.cap(x)

In [ ]:
ip_shape = (204800,)
inp = keras.Input(shape=ip_shape)
convs = []
for x in range(15):
    
    convs.append(binaryFeatureDetector()(inp))

out = layers.Concatenate()(convs)
conv_model = keras.Model(inputs=inp, outputs=out)

In [ ]:
model1 = keras.Sequential([
    res_model,
    layers.Flatten(),
    conv_model
])

In [ ]:
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE), optimizer=tf.keras.optimizers.Adadelta(learning_rate=1e-7, rho=(58/59)), metrics=['RootMeanSquaredError', 'binary_accuracy'])

In [ ]:
lastLoss = 999
overlap=1
for x in range(batches + overlap - 1):
    if (x >= batches):
        x = batches - 1
    for z in range(0, x):
        print(f'Starting Batch {z+1} out of {batches - 1}')
        model1, lastLoss = train_batch(z, model1, batch_length, 4, lastLoss, q=100., ls=4)

In [ ]:
for layer in model1.layers[2].layers[1:13]:
    layer.trainable = False

for layer in model1.layers[0].layers:
    layer.trainable = False

In [ ]:
val_x = []
val_y = []

val_data = assemble_batch(batches-1, batch_length)

for feature, labels in val_data:
    val_x.append(feature)
    val_y.append(labels[-1:])

del val_data    
        
for x in range(len(val_y)):
    val_y[x] = val_y[x].to_list()
    for y in range(len(val_y[x])):
        if (y == len(val_y[x]) - 1):
            val_y[x][y] = float(val_y[x][y])
        val_y[x][y] = float(val_y[x][y])
    val_y[x] = np.array(val_y[x])
        
val_x = np.array(val_x) / 255
val_x = preprocessdata(val_x)
    
val_y = np.asarray(val_y).astype('float32')

In [ ]:
wrapper = keras.Sequential([
    model1,
    ScoreRenderer(),
    layers.Dense(1, use_bias=False, input_shape=(None,2), kernel_constraint=tf.keras.constraints.UnitNorm(axis=0))
])

In [ ]:
wrapper.compile(loss=tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE), optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=(58/59)), metrics = ['RootMeanSquaredError'])

In [ ]:
lastLoss = 999
overlap=1
for x in range(batches + overlap - 1):
    if (x >= batches):
        x = batches - 1
    for z in range(0, x):
        print(f'Starting Batch {z+1} out of {batches - 1}')
        wrapper, lastLoss = train_batch(z, wrapper, batch_length, 4, lastLoss, f=-1, phase=2, ls=4)

In [ ]:
test_data = []
test_results = []
test_ids = []

for filename in os.listdir(path + 'test'):
    try:
        test_ids.append(filename[:-4])
        img_arr = cv2.imread(os.path.join(path + 'test', filename))[...,::-1] 
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
        test_data.append(resized_arr)
        test_data = np.array(test_data) / 255
        #test_data.reshape(-1, img_size, img_size, 1)
        test_data = preprocessdata(test_data)
        test_results.append(wrapper.predict(test_data).tolist()[0])
        test_data = []
        
    except Exception as e:
        print(e)
        
print(test_results)
print(test_ids)

In [ ]:
output_dict = {'Id': test_ids, 'Pawpularity': test_results}
output_df = pd.DataFrame(output_dict)
output_df.head()

In [ ]:
output_df.to_csv("submission.csv",index=False)